# COVID - 19

Questions:



Datasource: Google Cloud - Public Datasets, Covid19_usafacts

In [1]:
import pandas as pd

In [2]:
#create dataframe 
cases = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\confirmed_cases.csv')
deaths = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\deaths.csv')


In [3]:
#manipulate data: remove columns & unpivot
cases_drop = cases.drop(columns=['county_fips_code','county_name','state_fips_code'])
cases_unpivot = cases_drop.melt(id_vars=['state'], var_name='date', value_name='confirmed cases')
cases_unpivot['date'] = cases_unpivot.date.str[1:].replace('_', '-',regex=True)

deaths_drop = deaths.drop(columns=['county_fips_code','county_name','state_fips_code'])
deaths_unpivot = deaths_drop.melt(id_vars=['state'], var_name='date', value_name='deaths')
deaths_unpivot['date'] = deaths_unpivot.date.str[1:].replace('_', '-',regex=True)
deaths_unpivot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652239 entries, 0 to 652238
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   state   652239 non-null  object
 1   date    652239 non-null  object
 2   deaths  652239 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 14.9+ MB


In [ ]:
#merge dataframes
df = pd.merge(cases_unpivot, deaths_unpivot, on='date', how='left')


## Which top 5 states has the most covid cases?

In [ ]:
cases_unpivot.groupby('state').sum().sort_values('cases', ascending=False).head(5).round({'cases': 0}).astype(int).reset_index()

## Which state had the highest covid cases?

In [ ]:
cases_unpivot.max()

## Which state had the most covid cases?

In [ ]:
cases_unpivot.groupby('state').sum()